In [2]:
import pandas as pd
import os
import matplotlib.pyplot as plt
import traceback

In [11]:
import os
all_filesVU = []
all_filesANT = []
for i in [0, 1, 3, 5, 6]:
    folder = f'T{i}'
    try:
        folder_path = f'../Data/Activpal/Excel bestanden VUmc/{folder}'
        files_and_folders = os.listdir(folder_path)
        files = [os.path.join(folder_path, f) for f in files_and_folders if os.path.isfile(os.path.join(folder_path, f))]
        all_filesVU += files
    except:
        pass
    try:
        folder_path = f'../Data/Activpal/Excel bestanden Antonius/{folder}'
        files_and_folders = os.listdir(folder_path)
        files = [os.path.join(folder_path, f) for f in files_and_folders if os.path.isfile(os.path.join(folder_path, f))]
        all_filesANT += files
    except:
        pass

print(len(all_filesANT), len(all_filesVU))


143 664


In [ ]:
import pandas as pd
import re

def extract_patient_id(filename, prefix):
    """
    Extracts the patient ID using a regular expression that matches a specified prefix followed by two or three digits.
    Assumes this pattern is immediately followed by an underscore.
    Parameters:
        filename (str): The name of the file from which to extract the ID.
        prefix (str): The prefix to look for before the numeric ID.
    Returns:
        int: The extracted patient ID.
    Raises:
        ValueError: If the patient ID pattern is not found in the filename.
    """
    regex_pattern = rf'{re.escape(prefix)}(\d{{2,3}})_'
    match = re.search(regex_pattern, filename)
    if match:
        return int(match.group(1))
    else:
        raise ValueError(f"Patient ID not found in filename following the pattern {prefix}.")


import re

def extract_patient_ids(file_list):
    # Simplified regex pattern to capture any digits following 'VU' until the first non-digit character
    pattern = r'VU(\d+)'

    patient_ids = []
    for filename in file_list:
        match = re.search(pattern, filename)
        if match:
            patient_ids.append(match.group(1))
        else:
            print(f"Error processing {filename}: Patient ID not found.")
            patient_ids.append(None)
    
    return patient_ids





def process_file(file, prefix):
    """
    Reads an Excel file, extracts relevant columns, and appends additional info including a patient ID derived from the filename.
    Parameters:
        file (str): Path to the Excel file.
        prefix (str): Prefix to use in ID extraction from the filename.
    Returns:
        DataFrame: The processed data with additional columns for patient ID and time extracted from filename.
    """
    try:
        df = pd.read_excel(file, skiprows=13).iloc[:, [3, 5, 6, 7, 8, 10, 12, 16, 22, 23]]
        patid = extract_patient_id(file, prefix)
        tijd = file.split('/')[3][:2]
        
        df['patid'] = patid
        df['Tijd'] = tijd
        df['Locatie'] = prefix
        return df
    except Exception as e:
        print(f"Error processing {file}: {e}")
        return None

# Main loop to process files
dfs = []

for file in all_filesVU:
    df = process_file(file, 'VU')
    if df is not None:
        dfs.append(df)
for file in all_filesANT:
    df = process_file(file, 'ANT')
    if df is not None:
        dfs.append(df)

# Concatenate all DataFrames into one big DataFrame
df_AP = pd.concat(dfs, ignore_index=True)


Error processing ../Data/Activpal/Excel bestanden VUmc/T0\pp_VU24 T0-AP672460 202a 5Oct22 12-00am for 7d-CREA-PA08110875-DailySummaryOutcomes.xlsx: Patient ID not found in filename following the pattern VU.
Error processing ../Data/Activpal/Excel bestanden VUmc/T0\pp_VU25 T0-AP970369 202a 28Oct22 12-00am for 7d-CREA-PA08110875-DailySummaryOutcomes.xlsx: Patient ID not found in filename following the pattern VU.
Error processing ../Data/Activpal/Excel bestanden VUmc/T0\pp_VU27 T0-AP970371 202a 14Oct22 12-00am for 7d-CREA-PA08110875-DailySummaryOutcomes.xlsx: Patient ID not found in filename following the pattern VU.
Error processing ../Data/Activpal/Excel bestanden VUmc/T0\pp_VU28 T0-AP672459 202a 15Oct22 12-00am for 7d-CREA-PA08110875-DailySummaryOutcomes.xlsx: Patient ID not found in filename following the pattern VU.
Error processing ../Data/Activpal/Excel bestanden VUmc/T0\pp_VU29 T0-AP970370 202a 11Nov22 12-00am for 7d-CREA-PA08110875-DailySummaryOutcomes.xlsx: Patient ID not found

In [5]:
df_AP

,FileCode,DayOfWeek,Date,ValidDay,TotalTime(m),StepCount,TotalRLMsTime(m),TotalSedentaryTime(m),ActivityScore(MET.h),NumSitToStands,patid,Tijd,Locatie
0,pp_VU_01,Sat,2022-06-25 00:00:00,1.0,1440.0,10658.0,108.017,544.315,34.8889,47.0,../Data/Activpal/Excel bestanden VUmc/T0\pp_VU...,Ex,VU
1,pp_VU_01,Sun,2022-06-26 00:00:00,1.0,1440.0,8962.0,106.500,397.400,34.1000,48.0,../Data/Activpal/Excel bestanden VUmc/T0\pp_VU...,Ex,VU
2,pp_VU_01,Mon,2022-06-27 00:00:00,1.0,1440.0,6352.0,77.500,715.600,33.0000,60.0,../Data/Activpal/Excel bestanden VUmc/T0\pp_VU...,Ex,VU
3,pp_VU_01,Tue,2022-06-28 00:00:00,1.0,1440.0,11070.0,125.100,343.200,34.7000,76.0,../Data/Activpal/Excel bestanden VUmc/T0\pp_VU...,Ex,VU
4,pp_VU_01,Wed,2022-06-29 00:00:00,0.0,1440.0,2182.0,28.100,206.500,15.5000,29.0,../Data/Activpal/Excel bestanden VUmc/T0\pp_VU...,Ex,VU
...,...,...,...,...,...,...,...,...,...,...,...,...,...
5622,ANT26_T6,Fri,2024-03-01 00:00:00,1.0,1440.0,7330.0,125.700,545.800,34.4000,90.0,../Data/Activpal/Excel bestanden Antonius/T6\A...,Ex,ANT
5623,ANT26_T6,Sat,2024-03-02 00:00:00,0.0,1440.0,9144.0,106.100,462.100,18.3000,27.0,../Data/Activpal/Excel bestanden Antonius/T6\A...,Ex,ANT
5624,ANT26_T6,Sun,2024-03-03 00:00:00,0.0,1440.0,5030.0,76.000,491.300,18.7000,44.0,../Data/Activpal/Excel bestanden Antonius/T6\A...,Ex,ANT
5625,ANT26_T6,Mon,2024-03-04 00:00:00,0.0,1440.0,2548.0,43.900,750.000,19.7000,59.0,../Data/Activpal/Excel bestanden Antonius/T6\A...,Ex,ANT


In [5]:
import pandas as pd

# Assuming df is your DataFrame containing the data
df_AP = df_AP[df_AP['ValidDay'] == 1]

# Group by Locatie, patid, and Tijd and calculate averages and counts
grouped_data = df_AP.groupby(['Locatie', 'patid', 'Tijd']).agg({
    'StepCount': 'mean',
    'TotalRLMsTime(m)': 'mean',
    'TotalSedentaryTime(m)': 'mean',
    'ActivityScore(MET.h)': 'mean',
    'NumSitToStands': 'mean',
    'DayOfWeek': 'count'  # Count the number of observations
}).rename(columns={'DayOfWeek': 'Observations'})

grouped_data = grouped_data.reset_index()
grouped_data

import pandas as pd

# Assuming grouped_data is your DataFrame with columns 'Locatie' and 'patid'
# For demonstration, let's create a sample DataFrame

# Function to transform patid
def transform_patid(patid):
    if patid < 10:
        return '0' + str(patid)
    elif patid >= 100:
        return str(patid)
    else:
        return '{:02d}'.format(patid)

# Apply the function to patid
grouped_data['patid'] = grouped_data['patid'].apply(transform_patid)

# Creating the new column with the specified structure
grouped_data['pat_identifier'] = 'pp_OPRAH_' + grouped_data['Locatie'] + '_' + grouped_data['patid']
grouped_data

tijd = ['T0', 'T1', 'T3', 'T5', 'T6']

for t in tijd:
    df = grouped_data[grouped_data['Tijd'] == t]
    df.to_excel(f'../export/Activpal_{t}.xlsx')



In [7]:
grouped_data

,Locatie,patid,Tijd,StepCount,TotalRLMsTime(m),TotalSedentaryTime(m),ActivityScore(MET.h),NumSitToStands,Observations,pat_identifier
0,ANT,01,Ex,8686.000000,111.355000,633.797600,33.704640,62.800000,5,pp_OPRAH_ANT_01
1,ANT,02,Ex,4446.500000,64.713325,882.074750,32.441550,32.000000,8,pp_OPRAH_ANT_02
2,ANT,03,Ex,8280.800000,112.591000,577.808000,33.928440,60.000000,5,pp_OPRAH_ANT_03
3,ANT,04,Ex,10275.200000,137.280600,554.706000,34.831180,38.800000,5,pp_OPRAH_ANT_04
4,ANT,05,Ex,3085.000000,41.786675,650.832500,31.394150,29.250000,4,pp_OPRAH_ANT_05
...,...,...,...,...,...,...,...,...,...,...
182,VU,157,Ex,9258.000000,122.753833,666.775333,34.395367,65.166667,6,pp_OPRAH_VU_157
183,VU,158,Ex,6565.200000,92.752650,622.051750,33.787795,37.150000,20,pp_OPRAH_VU_158
184,VU,159,Ex,16352.476190,238.857095,545.278619,38.110924,68.476190,21,pp_OPRAH_VU_159
185,VU,160,Ex,6667.333333,93.607333,633.168333,33.025467,81.000000,3,pp_OPRAH_VU_160


In [8]:
# Extract unique 'patid' values for ANT and VU
patids_ANT = grouped_data.loc['ANT'].index.get_level_values('patid').unique()
patids_VU = grouped_data.loc['VU'].index.get_level_values('patid'). bv ()

# Get unique values of 'Tijd' from df_AP
unique_tijds = df_AP['Tijd'].unique()

# Create MultiIndex for ANT DataFrame
index_ANT = pd.MultiIndex.from_product([patids_ANT, unique_tijds], names=['patid', 'Tijd'])
df_ANT = pd.DataFrame(index=index_ANT, columns=grouped_data.columns).fillna(0)
df_ANT['Location'] = 'ANT'
df_ANT.set_index('Location', append=True, inplace=True)

# Create MultiIndex for VU DataFrame
index_VU = pd.MultiIndex.from_product([patids_VU, unique_tijds], names=['patid', 'Tijd'])
df_VU = pd.DataFrame(index=index_VU, columns=grouped_data.columns).fillna(0)
df_VU['Location'] = 'VU'
df_VU.set_index('Location', append=True, inplace=True)

# Concatenate ANT and VU DataFrames
empty_df = pd.concat([df_VU, df_ANT])

KeyError: 'ANT'

In [ ]:
# Add values from grouped_data and empty_df together
merged_df = empty_df.add(grouped_data, fill_value=0)

merged_df.to_excel('text.xlsx')
merged_df

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

# Sum up the 'Observations' column for each combination of 'patid', 'Locatie', and 'Tijd'
heatmap_data = grouped_data.groupby([ 'Locatie','patid', 'Tijd'])['Observations'].sum().unstack(fill_value=0)

# Voeg een extra kolom toe die het aantal kolommen telt waarvan de waarde niet gelijk aan 0 is
heatmap_data['AantalNietNul'] = [sum(1 for value in row if value != 0) for row in heatmap_data.values[:, :]]

heatmap_data


In [ ]:
import numpy as np
aantal_compleet = np.sum(heatmap_data['AantalNietNul'] == 5)
print(f'Er zijn {aantal_compleet} complete patienten')

In [ ]:
with pd.ExcelWriter('export/exportAP.xlsx') as writer:
    # Write grouped_data to the first sheet
    grouped_data.to_excel(writer, sheet_name='Gemiddeldes per patient')

    # Write heatmap_data to the second sheet
    heatmap_data.to_excel(writer, sheet_name='Aantal observation per patient')



### Initial Exploration

In [ ]:
# Calculate the total number of rows
total_days = len(df_AP)

# Calculate the number of valid days (where ValidDay == 1)
valid_days = df_AP[df_AP['ValidDay'] == 1]['ValidDay'].count()

# Calculate the ratio of valid days
valid_days_ratio = valid_days / total_days

print("Ratio of valid days:", round(100*valid_days_ratio, 1), "%")

In [ ]:
df_AP